Lesson 9: DE Analysis and Visualization

Now that you have a dds object, we'll:

Run differential expression

Explore results

Make plots:

Volcano plot

PCA plot

In [ ]:
#0 Check available levels
levels(dds$physiological_state)

In [ ]:
#✅ 1. Run DESeq and  extract results for a specific comparison:
# Example: compare UT2_D1nonSS vs UT2_D0.5
res_UT2_D1nonSS <- results(dds, contrast = c("physiological_state", "UT2_D1nonSS", "UT2_D0.5"))
summary(res_UT2_D1nonSS)


#You can repeat this for other conditions by changing "UT2_D1nonSS" to the condition of interest.

In [ ]:
# 2. Volcano plot
#Install the EnhancedVolcano package if not yet installed:


if (!requireNamespace("EnhancedVolcano", quietly = TRUE))
    BiocManager::install("EnhancedVolcano")
library(EnhancedVolcano)

#Then plot:
# Save to PDF
pdf("volcano_UT2_D1nonSSvsUT2_D05.pdf")
EnhancedVolcano(res_UT2_D1nonSS,
    lab = rownames(res_UT2_D1nonSS),
    x = "log2FoldChange",
    y = "pvalue",
    pCutoff = 0.05,
    FCcutoff = 1.0,
    title = "UT2_D1nonSS vs UT2_D0.5",
    subtitle = "Volcano plot of differential expression")
dev.off()



#You can repeat this for other conditions by changing "volcano_UT2_D1nonSSvsUT2_D05.pdf" 
#and "res_UT2_D1nonSS" and "UT2_D1nonSS vs UT2_D0.5"to the condition of interest.


In [ ]:
#Step 3: PCA plot
vsd <- vst(dds, blind = TRUE)

pcaData <- plotPCA(vsd, intgroup = "physiological_state", returnData = TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))

ggplot(pcaData, aes(PC1, PC2, color = physiological_state)) +
  geom_point(size = 4) +
  xlab(paste0("PC1: ", percentVar[1], "% variance")) +
  ylab(paste0("PC2: ", percentVar[2], "% variance")) +
  coord_fixed() +
  theme_minimal()



#The error :
#"Error in vst(dds, blind = TRUE) : less than 'nsub' rows, it is recommended to use varianceStabilizingTransformation directly
#means your count matrix (genes x samples) is smaller than the default minimum number of rows (nsub) expected by vst() for its internal gene selection step. This can happen if you have very few genes."

#Use varianceStabilizingTransformation() instead of vst() as recommended:
vsd <- varianceStabilizingTransformation(dds, blind = TRUE)

#and then PCA + using pdf() or png() device to capture plot:
png("PCA_plot.png", width=1200, height=900)
print(
  ggplot(pcaData, aes(PC1, PC2, color = physiological_state)) +
    geom_point(size = 4) +
    xlab(paste0("PC1: ", percentVar[1], "% variance")) +
    ylab(paste0("PC2: ", percentVar[2], "% variance")) +
    coord_fixed() +
    theme_minimal()
)
dev.off()

In [ ]:
#Step 4: Export results
res_UT2_D1nonSS_df <- as.data.frame(res_UT2_D1nonSS)
write.csv(res_UT2_D1nonSS_df, "statisticalandfunctionalanalysis/DEG_UT2_D1nonSSvsUT2_D05.csv")



#You can repeat this for other conditions by changing "res_UT2_D1nonSS_df" 
#and "res_UT2_D1nonSS" and "/DEG_UT2_D1nonSSvsUT2_D05.csv" to the condition of interest.